In [1]:
import requests
from bs4 import BeautifulSoup as BS
import datetime
import calendar
import pandas as pd
import os
dir_dof = 'D:/datos/dof'

In [2]:
def get_dof(fecha, edicion, incluye_texto=True):
    # Obtiene html
    url_base = 'https://www.dof.gob.mx'
    url = f'{url_base}/index.php?year={fecha.year}&month={fecha.month}&day={fecha.day}&edicion={edicion}'
    resp = requests.get(url=url)
    soup = BS(resp.content, 'html.parser')
    if fecha.year >= 2013:
        tag = 'a'
    else:
        tag = 'p'
        incluye_texto = False
    # selecciona tags que tienen la información
    tags = []
    for t in soup.select('td'):
        if t.attrs.get('class', None):
            if t.attrs['class'][0] in ['txt_blanco', 'txt_blanco2', 'subtitle_azul']:
                tags += [t]
        elif t.select_one(f'{tag}[class="enlaces"]'):
            if t.select_one(f'{tag}[class="enlaces"]').parent == t:
                tags += [t]
    # extrae la información de los tags
    poder, inst, texto, seccion = ('', '', '', '')
    entradas = []
    for t in tags:
        if t.attrs.get('class'):
            clase = t.attrs['class'][0]
            texto = t.text.strip()
            if clase=='txt_blanco':          
                seccion = texto
            elif clase=='txt_blanco2':
                    poder = texto
            elif clase=='subtitle_azul':
                inst = texto
        elif t.select_one(f'{tag}[class="enlaces"]'):
            e = t.select_one(f'{tag}[class="enlaces"]')
            if e.text.strip()!='Ver más':
                entradas += [{'fecha': fecha.isoformat(), 'edicion': edicion, 'seccion': seccion,
                              'poder': poder, 'inst': inst, 'titular': e.text.strip(), 'url_dof': url,
                              'url_texto': url_base+e['href'] if incluye_texto else '',
                              'texto': get_text_entrada_dof(url_base+e['href']) if incluye_texto else ''}]
    return entradas

def get_text_entrada_dof(url):
    resp = BS(requests.get(url).text, 'html.parser')
    for e in resp.select('style'):
        e.extract()
    text = resp.select_one('div[id="DivDetalleNota"]').text
    return text

def get_dof_rango(fecha_ini, fecha_fin, incluye_texto=True, ediciones = ['MAT', 'VES']):
    periodo = pd.date_range(start=fecha_ini, end=fecha_fin, freq='D')
    dof = []
    for dia in periodo:
        for ed in ediciones:
            try:
                dof += get_dof(dia.date(), ed, incluye_texto=incluye_texto)
            except:
                print(f'error en {dia} edicion {ed}')
    return dof

def get_dof_year_mes(year, mes, incluye_texto=True, ediciones = ['MAT', 'VES']):
    dof = []
    for dia in calendar.Calendar().itermonthdates(year, mes):
        if (dia <= datetime.date.today()) and (dia.month==mes):
            for ed in ediciones:
                try:
                    dof += get_dof(dia, ed, incluye_texto=incluye_texto)
                except:
                    print(f'error en {dia} edicion {ed}')
    return dof

## una edición del dof

In [ ]:
# Edición vespertina del 17 de mayo
fecha = datetime.date(2019, 5, 17)
dof = get_dof(fecha=fecha, edicion='VES')

In [12]:
df = pd.DataFrame(dof).head().rename(columns={'titular': 'titulo', 'poder': 'rama'})
df.head()[['fecha', 'edicion', 'seccion', 'rama', 'inst', 'titulo', 'texto', 'url_dof', 'url_texto']]

,fecha,edicion,seccion,rama,inst,titulo,texto,url_dof,url_texto
0,2019-05-02,MAT,PRIMERA SECCION,PODER EJECUTIVO,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,Convenio de Coordinación que en el marco del S...,\n CONVENIO de Coordinación que en el marco ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
1,2019-05-02,MAT,PRIMERA SECCION,PODER EJECUTIVO,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,Convenio de Coordinación que en el marco del S...,\n CONVENIO de Coordinación que en el marco ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
2,2019-05-02,MAT,PRIMERA SECCION,PODER EJECUTIVO,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,Convenio de Coordinación que en el marco del S...,\n CONVENIO de Coordinación que en el marco ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
3,2019-05-02,MAT,PRIMERA SECCION,PODER EJECUTIVO,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,Convenio Específico de Adhesión para el otorga...,\n CONVENIO Específico de Adhesión para el o...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
4,2019-05-02,MAT,PRIMERA SECCION,PODER EJECUTIVO,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,Declaratoria de Emergencia por la presencia de...,\n DECLARATORIA de Emergencia por la presenc...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...


## Todos los dof en un rango de fechas

In [13]:
# Todos los dof entre el 14 y el 17 de mayo, solo edicion vespertina
fecha_ini = datetime.date(2019, 5, 14)
fecha_fin = datetime.date(2019, 5, 17)
dof = get_dof_rango(fecha_ini=fecha_ini, fecha_fin=fecha_fin, ediciones=['VES'])
pd.DataFrame(dof).tail()

,edicion,fecha,inst,poder,seccion,texto,titular,url_dof,url_texto
11,VES,2019-05-17,PRESIDENCIA DE LA REPUBLICA,PODER EJECUTIVO,UNICA SECCION,\n DECRETO por el que se crea el Consejo Nac...,Decreto por el que se crea el Consejo Nacional...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
12,VES,2019-05-17,SECRETARIA DE GOBERNACION,PODER EJECUTIVO,UNICA SECCION,\n DECRETO por el que se deroga el diverso p...,Decreto por el que se deroga el diverso por el...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
13,VES,2019-05-17,SECRETARIA DE HACIENDA Y CREDITO PUBLICO,PODER EJECUTIVO,UNICA SECCION,\n DECLARATORIA por la que se establece que ...,Declaratoria por la que se establece que el in...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
14,VES,2019-05-17,SECRETARIA DE AGRICULTURA Y DESARROLLO RURAL,PODER EJECUTIVO,UNICA SECCION,\n CONVENIO de Coordinación para el Desarrol...,Convenio de Coordinación para el Desarrollo Ru...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
15,VES,2019-05-17,SECRETARIA DE AGRICULTURA Y DESARROLLO RURAL,PODER EJECUTIVO,UNICA SECCION,\n CONVENIO de Coordinación para el Desarrol...,Convenio de Coordinación para el Desarrollo Ru...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...


## Todos los dof de un mes

In [78]:
# Todos los dof de 2012 a 1918, por mes
for year in range(1917, 1919):
    for mes in range(1, 13):
        print(year, mes)
        dof = get_dof_year_mes(year, mes)
        df = pd.DataFrame(dof)
        dir_dof = 'D:/datos/dof'
        df.to_csv(f'{dir_dof}/dof_{year}_{mes}.csv', index=False)

1917 1
1917 2
1917 3
1917 4
1917 5
1917 6
1917 7
1917 8
1917 9
1917 10
1917 11
1917 12
1918 1
1918 2
1918 3
1918 4
1918 5
1918 6
1918 7
1918 8
1918 9
1918 10
1918 11
1918 12


In [4]:
# Todos los dof de 2013 a 2018, por mes
for year in range(2013, 2019):
    for mes in range(1, 13):
        print(year, mes)
        dof = get_dof_year_mes(year, mes)
        df = pd.DataFrame(dof)
        dir_dof = 'D:/datos/dof'
        df.to_csv(f'{dir_dof}/dof_{year}_{mes}.csv', index=False)

2013 1
2013 2
2013 3
2013 4
2013 5
2013 6
2013 7
2013 8
2013 9
2013 10
2013 11
2013 12
2014 1
2014 2
2014 3
2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2014 11
2014 12
2015 1
2015 2
2015 3
2015 4
2015 5
2015 6
2015 7
2015 8
2015 9
2015 10
2015 11
2015 12
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12


In [3]:
# Todos los dof de enero a abril de 2019, por mes
year = 2019
for mes in range(1, 11):
    filedof = f'{dir_dof}/dof_{year}_{mes}.csv'
    if not os.path.exists(filedof):
        print(year, mes)
        dof = get_dof_year_mes(year, mes)
        df = pd.DataFrame(dof)
        dir_dof = 'D:/datos/dof'
        df.to_csv(f'{dir_dof}/dof_{year}_{mes}.csv', index=False)

2019 5
2019 6
2019 7
2019 8
2019 9
2019 10


## Exportando bases

### Exportando base de 1917 a 2012

In [7]:
df = pd.concat([pd.read_csv(f'{dir_dof}/dof_{y}_{m}.csv', encoding='utf-8') for y in range(1917, 2013) for m in range(1, 13)])\
    .rename(columns={'poder': 'rama', 'titular': 'titulo'})
df.to_csv(dir_dof+'/dof_1917_2012.csv', index=False, encoding='utf-8')

### Exportando base anual de 2013 a 2019

In [4]:
for y in range(2013, 2020):
    dof_year_name = f'{dir_dof}/dof_{y}.csv'
    if not os.path.exists(dof_year_name):
        df = pd.concat([pd.read_csv(f'{dir_dof}/dof_{y}_{m}.csv', encoding='utf-8') for m in range(1, 13) if os.path.exists(f'{dir_dof}/dof_{y}_{m}.csv')])\
            .rename(columns={'poder': 'rama', 'titular': 'titulo'})
        df.to_csv(dof_year_name, index=False, encoding='utf-8')

In [5]:
dof2019 = pd.read_csv(dof_year_name)

In [8]:
dof2019.head()

,edicion,fecha,inst,rama,seccion,texto,titulo,url_dof,url_texto
0,MAT,2019-01-02,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,PODER EJECUTIVO,UNICA SECCION,\n DECLARATORIA de Emergencia Extraordinar...,Declaratoria de Emergencia Extraordinaria por ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
1,MAT,2019-01-02,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,PODER EJECUTIVO,UNICA SECCION,\n DECLARATORIA de Emergencia Extraordinar...,Declaratoria de Emergencia Extraordinaria por ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
2,MAT,2019-01-02,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,PODER EJECUTIVO,UNICA SECCION,\n DECLARATORIA de Emergencia Extraordinar...,Declaratoria de Emergencia Extraordinaria por ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
3,MAT,2019-01-02,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,PODER EJECUTIVO,UNICA SECCION,\n DECLARATORIA de Emergencia Extraordinar...,Declaratoria de Emergencia Extraordinaria por ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
4,MAT,2019-01-02,SECRETARIA DE SEGURIDAD Y PROTECCION CIUDADANA,PODER EJECUTIVO,UNICA SECCION,\n DECLARATORIA de Emergencia Extraordinar...,Declaratoria de Emergencia Extraordinaria por ...,https://www.dof.gob.mx/index.php?year=2019&mon...,https://www.dof.gob.mx/nota_detalle.php?codigo...
